In [2]:
import os
import cv2
import numpy as np

def preprocess_signature(image_path, size=(256, 256)):
    """Preprocessing gambar tanda tangan sebelum digunakan dalam model"""
    # Convert to grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  
    
    # Resize gambar
    image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)  
    
    # Normalisasi dan thresholding
    image = image.astype(np.float32) / 255.0
    image_uint8 = (image * 255).astype(np.uint8)
    _, image = cv2.threshold(image_uint8, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Pastikan latar belakang putih dan tanda tangan hitam
    if np.mean(image) < 127:  # Invert gambar jika perlu
        image = cv2.bitwise_not(image)
    
    # Ekstrak kontur dan fokus pada tanda tangan
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        image = image[y:y+h, x:x+w]  # Cropping
    
    # Inversi background (background hitam, tanda tangan putih)
    inverted_image = cv2.bitwise_not(image)
    
    return (image / 255.0).astype(np.float32), (inverted_image / 255.0).astype(np.float32)

def augment_image(image):
    augmented_images = []

    # PENCERAHAN GAMBAR (Brightness Adjustment)
    def brighten_image(img, alpha=1.5, beta=40):
        return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    
    brightened = brighten_image(image)
    augmented_images.append(brightened)

    # VARIASI KONTRAS (Kontras lebih tinggi atau rendah)
    contrast_high = brighten_image(image, alpha=2.0, beta=0)
    contrast_low = brighten_image(image, alpha=0.8, beta=0)
    augmented_images.append(contrast_high)
    augmented_images.append(contrast_low)

    # DILATION & EROSION (Tapi lebih ringan dari sebelumnya)
    kernel = np.ones((2,2), np.uint8)
    dilated = cv2.dilate(image, kernel, iterations=1)
    eroded = cv2.erode(image, kernel, iterations=1)
    augmented_images.append(dilated)
    augmented_images.append(eroded)

    # RESIZE VARIATIONS
    resized_small = cv2.resize(image, (128, 128), interpolation=cv2.INTER_AREA)
    resized_large = cv2.resize(image, (512, 512), interpolation=cv2.INTER_LINEAR)
    augmented_images.append(resized_small)
    augmented_images.append(resized_large)

    # GAUSSIAN NOISE (Simulasi blur/gangguan sensor)
    def add_gaussian_noise(img, mean=0, sigma=20):
        noise = np.random.normal(mean, sigma, img.shape).astype(np.uint8)
        noisy_image = cv2.add(img, noise)
        return noisy_image

    gaussian_noisy = add_gaussian_noise(image)
    augmented_images.append(gaussian_noisy)

    # SALT & PEPPER NOISE (Simulasi bercak acak)
    def add_salt_and_pepper(img, salt_prob=0.02, pepper_prob=0.02):
        noisy_img = img.copy()
        total_pixels = img.size
        num_salt = int(total_pixels * salt_prob)
        num_pepper = int(total_pixels * pepper_prob)

        # Tambahkan salt (titik putih)
        coords = [np.random.randint(0, i - 1, num_salt) for i in img.shape]
        noisy_img[coords[0], coords[1]] = 255

        # Tambahkan pepper (titik hitam)
        coords = [np.random.randint(0, i - 1, num_pepper) for i in img.shape]
        noisy_img[coords[0], coords[1]] = 0

        return noisy_img

    salt_pepper_noisy = add_salt_and_pepper(image)
    augmented_images.append(salt_pepper_noisy)

    return augmented_images

def preprocess_all_images(input_folder, output_folder):
    """Proses semua gambar di folder input, simpan hasil di folder output"""
    os.makedirs(output_folder, exist_ok=True)
    
    for folder_name in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder_name)
        
        if os.path.isdir(folder_path):
            first_name = folder_name.split()[0]  # Extract first name
            destination_path = os.path.join(output_folder, folder_name)
            os.makedirs(destination_path, exist_ok=True)
            
            for idx, file_name in enumerate(os.listdir(folder_path), start=1):
                source_file = os.path.join(folder_path, file_name)
                new_file_name = f"{first_name}_{idx}.jpg"
                destination_file = os.path.join(destination_path, new_file_name)
                destination_file_inverted = os.path.join(destination_path, f"inverted_{new_file_name}")
                
                if os.path.isfile(source_file):
                    processed_image, inverted_image = preprocess_signature(source_file)
                    if processed_image is not None:
                        cv2.imwrite(destination_file, (processed_image * 255).astype(np.uint8))
                        cv2.imwrite(destination_file_inverted, (inverted_image * 255).astype(np.uint8))
                        
                        # Apply augmentasi dan simpan gambar baru
                        augmented_images = augment_image((processed_image * 255).astype(np.uint8))
                        for i, aug_img in enumerate(augmented_images):
                            cv2.imwrite(os.path.join(destination_path, f"aug_{i}_{new_file_name}"), aug_img)
                        
                        print(f"Processed and saved {new_file_name} and its augmented versions to {destination_path}")

# Contoh penggunaan
train_directory = "train"
test_directory = "test"
processed_train_directory = "processed_train"
processed_test_directory = "processed_test"
preprocess_all_images(train_directory, processed_train_directory)
preprocess_all_images(test_directory, processed_test_directory)
print("Semua preprocessing selesai.")

Processed and saved Lithania_1.jpg and its augmented versions to processed_train/Lithania
Processed and saved Lithania_2.jpg and its augmented versions to processed_train/Lithania
Processed and saved Lithania_3.jpg and its augmented versions to processed_train/Lithania
Processed and saved Lithania_4.jpg and its augmented versions to processed_train/Lithania
Processed and saved Lia_1.jpg and its augmented versions to processed_train/Lia Kurniawati
Processed and saved Lia_2.jpg and its augmented versions to processed_train/Lia Kurniawati
Processed and saved Lia_3.jpg and its augmented versions to processed_train/Lia Kurniawati
Processed and saved Lia_4.jpg and its augmented versions to processed_train/Lia Kurniawati
Processed and saved Mariati_1.jpg and its augmented versions to processed_train/Mariati unidayan
Processed and saved Mariati_2.jpg and its augmented versions to processed_train/Mariati unidayan
Processed and saved Mariati_3.jpg and its augmented versions to processed_train/Ma